In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Example text to classify
texts = ["I love this product!", "This is terrible, do not buy it."]

# Tokenize the texts
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Fine-tuning and classification would happen here with proper datasets
outputs = model(**inputs)


c:\Users\mubasshirahmxd\anaconda3\envs\fsds_dl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
!pip install datasets

In [3]:
#!pip install datasets # Install datasets if not already installed
import datasets # Import the datasets library into your current environment
datasets.__version__ # Access the version of the datasets package

'4.4.1'

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("imdb")

# Load the pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate every epoch
    learning_rate=2e-5,              # learning rate for optimization
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    num_train_epochs=3,              # number of epochs to train
    weight_decay=0.01,               # strength of weight decay (regularization)
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the model to train
    args=training_args,                  # training arguments
    train_dataset=tokenized_datasets['train'],  # training dataset
    eval_dataset=tokenized_datasets['test'],   # evaluation dataset
)

# Start the training process
trainer.train()

Map:  56%|█████▌    | 14000/25000 [00:40<00:31, 344.50 examples/s]


KeyboardInterrupt: 

In [10]:
import torch # Import the correct PyTorch module
# After fine-tuning, you can use the model for predictions
inputs = tokenizer(["I love this product!", "This is terrible, do not buy it."], padding=True, truncation=True, return_tensors="pt")

# Ensure model is in evaluation mode
model.eval()

# Move inputs and model to the same device (e.g., CUDA if available)
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device) # Move the model to the device
    inputs = inputs.to(device) # Move the inputs to the device
else:
    device = torch.device("cpu")

# Get predictions
with torch.no_grad(): # Use torch.no_grad() for inference
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probs = torch.nn.functional.softmax(logits, dim=-1) # Use torch.nn.functional.softmax()

# Get predicted class (e.g., 0 or 1 for binary classification)
predictions = torch.argmax(probs, dim=-1) # Use torch.argmax()
print(predictions)

tensor([1, 0], device='cuda:0')


In [11]:
# Evaluate the model on the test set
eval_results = trainer.evaluate()

# Print evaluation results
print(eval_results)


KeyboardInterrupt: 

In [ ]:
# Save the trained model
model.save_pretrained('./bert_imdb_model')
tokenizer.save_pretrained('./bert_imdb_model')


In [ ]:
# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('./bert_imdb_model')
tokenizer = BertTokenizer.from_pretrained('./bert_imdb_model')

# Function to make predictions
def predict(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(axis=-1)
    return predictions

# Example prediction
texts = ["I love this movie!", "This movie was terrible."]
predictions = predict(texts)

print(predictions)


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class TextRequest(BaseModel):
    text: str

@app.post("/predict/")
def predict(request: TextRequest):
    inputs = tokenizer(request.text, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    prediction = outputs.logits.argmax(axis=-1).item()
    return {"prediction": prediction}

# Run the FastAPI app
# uvicorn app:app --reload
